In [1]:
import pandas as pd
import numpy as np
import collections
import copy
import random
import re
import matplotlib.pyplot as plt
from nlp.utils import (
    plot_histogram,
    get_completetext,
    plot_wordcloud,
    print_statistics,
    groups_frequency_sort)
from nlp.text_statistics import (
    count_tokens,
    unique_tokens
)
from utils.read_files import (
    get_items)
from item.item_list import (
    ItemList,
    Item
)
from item.utils import get_tokens_set
import seaborn as sns

sns.set()

# Load price statistics

In [2]:
prices = pd.read_csv('../data/output/druid_fasttext/cluster_prices_statistics.csv.zip', sep=';')
items = pd.read_csv("../data/output/druid_fasttext/items_clusters_train_wo_out.csv.zip", sep=';')

In [3]:
len(prices[['dsc_unidade_medida', 'grupo']].drop_duplicates())

294893

In [5]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294893 entries, 0 to 294892
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   grupo               294893 non-null  object 
 1   dsc_unidade_medida  294893 non-null  object 
 2   media               294893 non-null  float64
 3   qtd                 294893 non-null  int64  
 4   max                 294893 non-null  float64
 5   min                 294893 non-null  float64
 6   mediana             294893 non-null  float64
 7   desvio_padrao       210174 non-null  float64
 8   var                 210174 non-null  float64
 9   quantil_1           294893 non-null  float64
 10  quantil_3           294893 non-null  float64
 11  primeiro_termo      294856 non-null  object 
 12  grupo_id            294893 non-null  int64  
 13  ruido               294893 non-null  int64  
dtypes: float64(8), int64(3), object(3)
memory usage: 31.5+ MB


In [7]:
prices['first_token'] = prices['grupo'].str.split('_').str[0]

In [8]:
prices['desvio_padrao'] = prices['desvio_padrao'].fillna(0.0)
prices['var'] = prices['var'].fillna(0.0)
prices['media'] = prices['media'].fillna(1.0)

In [9]:
prices = prices.rename({'qtd': 'n_items'}, axis=1)

In [10]:
prices.head()

,grupo,dsc_unidade_medida,media,n_items,max,min,mediana,desvio_padrao,var,quantil_1,quantil_3,primeiro_termo,grupo_id,ruido,first_token
0,00a,unidade,1533.3333,1,1533.3333,1533.3333,1533.3333,0.000000,0.000000,1533.3333,1533.3333,00a,0,1,00a
1,00m,unidade,0.5270,2,0.6000,0.4540,0.5270,0.103238,0.010658,0.4905,0.5635,00m,1,1,00m
2,0dx,pacote,2.7300,1,2.7300,2.7300,2.7300,0.000000,0.000000,2.7300,2.7300,0dx,2,1,0dx
3,0to,peca,1.0000,1,1.0000,1.0000,1.0000,0.000000,0.000000,1.0000,1.0000,0to,3,1,0to
4,0to,unidade,15000.0000,1,15000.0000,15000.0000,15000.0000,0.000000,0.000000,15000.0000,15000.0000,0to,4,1,0to


In [11]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8746361 entries, 0 to 8746360
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   grupo               object 
 1   dsc_unidade_medida  object 
 2   grupo_id            int64  
 3   item_id             int64  
 4   grupo_ruido         int64  
 5   item_ruido          int64  
 6   preco               float64
 7   original            object 
 8   licitacao           int64  
 9   ano                 int64  
 10  original_dsc        object 
 11  primeiro_termo      object 
dtypes: float64(1), int64(6), object(5)
memory usage: 800.8+ MB


In [20]:
# remove outliers
items = items[items.item_ruido != 1]
prices = prices[prices.ruido != 1]

In [13]:
items.head()

,grupo,dsc_unidade_medida,grupo_id,item_id,grupo_ruido,item_ruido,preco,original,licitacao,ano,original_dsc,primeiro_termo
91,a100_0,caixa,30,5814772,0,0,48.355,SERINGA DE RESINA Z100 MICROPARTICULAS A1,204107,2017,seringa resina a100 microparticulas a1,a100
92,a100_0,tubo,31,1727657,0,0,45.130,RESINA Z100 A1,587025,2014,resina a100 a1,a100
93,a100_0,tubo,31,5796907,0,0,51.910,RESINA Z100 - A1,267537,2017,resina a100 a1,a100
94,a100_0,tubo,31,6186318,0,0,44.860,RESINA Z100 A1,587149,2017,resina a100 a1,a100
95,a100_0,tubo,31,9738429,0,0,56.500,RESINA Z100 - A1,585858,2019,resina a100 a1,a100


In [14]:
items_df = pd.merge(items, prices.drop('first_token', axis=1), how='left', left_on=['grupo','dsc_unidade_medida'], right_on = ['grupo','dsc_unidade_medida'])

## Print clusters info

In [18]:
def print_clusters_info(cluster_name, min_size=20, total_unit_metrics=5, total_clusters=5, total_items=5, total_words=20):
    
    clusters = prices[prices.first_token == cluster_name][['grupo', 'dsc_unidade_medida', 'n_items']]
    clusters_ranking = clusters.sort_values(by='n_items', ascending=False)
    
    print("#itens:", sum(list(clusters["n_items"])))
    print("#grupos:", len(clusters_ranking))
    print("Grupos com +20 itens:", len(clusters_ranking[clusters_ranking.n_items >= min_size]))
    print("#itens/#grupos: {:.2f}".format(sum(list(clusters["n_items"]))/len(clusters_ranking)))

    top_unit_metrics = clusters.groupby("dsc_unidade_medida").sum().sort_values(by='n_items', ascending=False).head(total_unit_metrics).reset_index()
    
    print()
    print("Unidades de medida mais frequentes:")
    for unit in top_unit_metrics.iterrows():
        print("{} ({:.1f}%)".format(unit[1]['dsc_unidade_medida'], (unit[1]['n_items']/sum(list(clusters["n_items"])))*100))
    print()
        
    clusters_ranking = clusters_ranking[clusters_ranking.n_items >= min_size].head(total_clusters)
    
    for cluster_id in clusters_ranking.index.values:
        cluster_info = clusters.loc[cluster_id]
        cluster_items = items[(items.grupo == cluster_info['grupo']) & (items.dsc_unidade_medida == cluster_info['dsc_unidade_medida'])].copy()
        cluster_items["description_tokens"] = cluster_items.original_dsc.apply(lambda d: d.split())
        top_descriptions = cluster_items.groupby("original_dsc").size().sort_values(ascending=False).head(total_items)
        present_tokens = collections.Counter()
        for tokens in cluster_items["description_tokens"]:
            present_tokens.update(tokens)

        present_tokens = list(present_tokens.items())
        present_tokens.sort(key= lambda x : x[1], reverse=True)
        present_tokens = [token[0] for token in present_tokens][:total_words]

        print("Cluster {} ({}): ".format(cluster_info['grupo'], cluster_info['dsc_unidade_medida']))
        print("Tamanho: {}".format(len(cluster_items)))
        print("Descrições únicas: {}".format(len(set(cluster_items["original_dsc"]))))
        print("{} Termos mais frequentes: {}".format(total_words, present_tokens))
        print("Descrições mais frequentes:")
        for desc in top_descriptions.iteritems():
            print("{} ({:.1f}%)".format(desc[0], (desc[1]/len(cluster_items))*100))
        print("\n")

# Abobora

In [17]:
print_clusters_info("abobora")

#itens: 3674
#grupos: 49
Grupos com +20 itens: 19
#itens/#grupos: 74.98

Unidades de medida mais frequentes:
kg (95.3%)
unidade (3.5%)
kg grama (0.4%)
kg 1 kg (0.2%)
md (0.2%)

Cluster abobora_11 (kg): 
Tamanho: 1138
Descrições únicas: 25
20 Termos mais frequentes: ['abobora', 'madurar', 'abater', 'verde', 'tipo', 'cambodia', 'cambuta', 'mogangar', 'cathia', 'sem', 'apresentar', 'paulistinha', 'princesa', 'sergipano', 'cp', 'tamanho', 'medio', 'r', 'caban', 'cambucha']
Descrições mais frequentes:
abobora (44.9%)
abobora madurar (31.0%)
abobora abater (12.7%)
abobora verde (4.8%)
abobora tipo abater (1.1%)


Cluster abobora_15 (kg): 
Tamanho: 857
Descrições únicas: 275
20 Termos mais frequentes: ['abobora', 'sem', 'tamanho', 'casca', 'moranga', 'firmar', 'qualidade', 'madurar', 'com', 'uniforme', 'primeiro', 'coloracao', 'apresentar', 'defeito', 'sujidade', 'livre', 'medio', 'japones', 'grau', 'consumo']
Descrições mais frequentes:
abobora moranga japones casca dever ser liso sem mancha

# Dipirona

In [29]:
print_clusters_info("dipirona")

#itens: 3379
#grupos: 321
Grupos com +20 itens: 44
#itens/#grupos: 10.53

Unidades de medida mais frequentes:
unidade (26.0%)
frasco (23.4%)
comprimido (19.1%)
ampola (12.2%)
amp (3.9%)

Cluster dipirona_9 (comprimido): 
Tamanho: 327
Descrições únicas: 22
20 Termos mais frequentes: ['dipirona', '500', 'mg', 'comprimido', 'comp', 'compr', 'cpr', 'cp', '129', '8974', '4731', '345', '0017', '959', '9', 'un', '966', 'epp', 'medic', 'fracionar']
Descrições mais frequentes:
dipirona 500 mg (60.6%)
dipirona 500 mg comprimido (15.0%)
dipirona 500 mg comp (7.3%)
dipirona 500 mg compr (3.1%)
dipirona 500 mg cpr (2.1%)


Cluster dipirona_2 (comprimido): 
Tamanho: 248
Descrições únicas: 18
20 Termos mais frequentes: ['dipirona', 'sodico', 'mg', '500', 'comprimido', 'comp', 'compr', 'cpr', 'cp', '9', '5', '250', 'com', 'ct', 'bl', 'al', 'plas', 'inc', 'x', '240']
Descrições mais frequentes:
dipirona sodico 500 mg (60.9%)
dipirona sodico 500 mg comprimido (23.0%)
dipirona sodico 500 mg comp (3.6%)
d

In [ ]:
dipirona = items_df[items_df.first_token == 'dipirona']

In [ ]:
set(dipirona[(dipirona.cluster == 'dipirona_9') & (dipirona.dsc_unidade_medida == 'comprimido')]['description'])

In [ ]:
dipirona[(dipirona.cluster == 'dipirona_0') & (dipirona.dsc_unidade_medida == 'ampola')]

# Pneu

In [15]:
print_clusters_info("pneu")

#itens: 43417
#grupos: 660
Grupos com +20 itens: 338
#itens/#grupos: 65.78

Unidades de medida mais frequentes:
unidade (95.0%)
peca (2.9%)
unidades (1.4%)
serv (0.2%)
pca (0.1%)

Cluster pneu_46 (unidade): 
Tamanho: 11916
Descrições únicas: 1997
20 Termos mais frequentes: ['pneu', 'r', '70', '75', '16', '15', '5', '205', 'x', 'r15', 'r14', '185', '225', '17', '215', '80', '65', 'r16', '195', '20']
Descrições mais frequentes:
pneu 175 70 r14 (2.9%)
pneu 215 75 r 17 5 (2.6%)
pneu 215 75 17 5 (2.4%)
pneu 205 70 r15 (1.9%)
pneu 205 70 r 15 (1.9%)


Cluster pneu_44 (unidade): 
Tamanho: 1868
Descrições únicas: 286
20 Termos mais frequentes: ['pneu', '14', 'r', '70', '175', '185', 'x', '65', 'c', '9', 'radial', 'lona', 't', '24', '28', '80', '8', '88', '20', '60']
Descrições mais frequentes:
pneu 175 70 r 14 (14.8%)
pneu 175 70 14 (7.9%)
pneu 185 r 14 (6.4%)
pneu 185 70 r 14 (6.3%)
pneu 175 65 r 14 (4.2%)


Cluster pneu_188 (unidade): 
Tamanho: 1727
Descrições únicas: 409
20 Termos mais freq

In [ ]:
pneu = items_df[items_df.first_token == 'pneu']

In [ ]:
set(pneu[(pneu.cluster == 'pneu_46') & (pneu.dsc_unidade_medida == 'unidade')]['description'])

In [ ]:
pneu[(pneu.cluster == 'pneu_1') & (pneu.dsc_unidade_medida == 'unidade')]

# Gasolina

In [21]:
print_clusters_info("gasolina")

#itens: 5448
#grupos: 35
Grupos com +20 itens: 17
#itens/#grupos: 155.66

Unidades de medida mais frequentes:
litro (97.2%)
unidade (2.3%)
litro s (0.1%)
21 (0.1%)
gab pref (0.1%)

Cluster gasolina_8 (litro): 
Tamanho: 3284
Descrições únicas: 34
20 Termos mais frequentes: ['gasolina', 'comum', 'automotivo', 'tipo', 'c', 'aditivado', 'aut', 'm', 'v', 'epp', 'r1', 'r', 'comumgasolina', 'combust', 'incolor', 'amarelar', 'social', 'amarelo', 'assist', 'r2']
Descrições mais frequentes:
gasolina comum (90.9%)
gasolina automotivo comum (3.7%)
gasolina automotivo tipo comum (1.1%)
gasolina c comum (0.7%)
gasolina comum automotivo (0.6%)


Cluster gasolina_5 (litro): 
Tamanho: 945
Descrições únicas: 20
20 Termos mais frequentes: ['gasolina', 'aditivado', 'automotivo', 'comun', 'c', 'tipo', 'activada', 'comuma', 'cras', 'sec', 'prom', 'social', 'n', 'common', 'montalvania', 'adm', 'f', 'grid']
Descrições mais frequentes:
gasolina (61.0%)
gasolina aditivado (19.2%)
gasolina comun (6.9%)
gasolina 

# Máscara

In [32]:
print_clusters_info("mascara")

#itens: 8007
#grupos: 349
Grupos com +20 itens: 87
#itens/#grupos: 22.94

Unidades de medida mais frequentes:
unidade (58.7%)
caixa (29.2%)
pacote (4.0%)
peca (1.8%)
kit (1.6%)

Cluster mascara_26 (caixa): 
Tamanho: 1304
Descrições únicas: 351
20 Termos mais frequentes: ['mascara', 'descartavel', 'elastico', '50', 'c', 'com', 'cirurgico', 'unidade', 'caixa', 'cx', '100', 'desc', 'unid', 'branco', 'descartar', 'und', 'un', 'tnt', 'cor', 'descarar']
Descrições mais frequentes:
mascara descartavel (11.4%)
mascara cirurgico descartavel (7.4%)
mascara cirurgico (2.5%)
mascara descartavel com elastico (2.5%)
mascara descartavel c elastico (2.4%)


Cluster mascara_26 (unidade): 
Tamanho: 925
Descrições únicas: 312
20 Termos mais frequentes: ['mascara', 'descartavel', 'com', 'cirurgico', 'elastico', '50', 'a95', 'n', '95', 'c', 'nebulizacao', 'unidade', 'pff', 'p', 'azul', 'respirador', 'respiratorio', 'caixa', 'facial', 'venturi']
Descrições mais frequentes:
mascara descartavel (9.2%)
mascara

# Locação

In [34]:
print_clusters_info("locacao")

#itens: 10733
#grupos: 1596
Grupos com +20 itens: 122
#itens/#grupos: 6.72

Unidades de medida mais frequentes:
unidade (38.1%)
diaria (11.6%)
hora (7.5%)
sv (3.4%)
mes (3.0%)

Cluster locacao_38 (unidade): 
Tamanho: 207
Descrições únicas: 18
20 Termos mais frequentes: ['banheiro', 'quimico', 'locacao', 'unidade', 'individual', 'equipamento', '978', '3939', 'polietileno', 'portatil', 'para', 'p', 'n', 'adaptado', 'carnaval', 'unid', 'plastico', 'def', 'deficiente', '4183']
Descrições mais frequentes:
locacao banheiro quimico (76.8%)
locacao banheiro quimico unidade (4.8%)
locacao banheiro quimico individual (2.4%)
locacao banheiro quimico 3939 (1.9%)
locacao banheiro quimico 978 (1.9%)


Cluster locacao_22 (unidade): 
Tamanho: 194
Descrições únicas: 50
20 Termos mais frequentes: ['locacao', 'tenda', 'x', '6', 'm', '4', '5', 'barraca', '8', 'piramidal', 'lona', 'unidade', 'tipo', 'mts', 'tamanho', 'i', 'tensionado', 'mx', 'matalon', 'medir']
Descrições mais frequentes:
locacao tenda (25

# Veículo

In [35]:
print_clusters_info("veiculo")

#itens: 3722
#grupos: 180
Grupos com +20 itens: 44
#itens/#grupos: 20.68

Unidades de medida mais frequentes:
unidade (79.2%)
sv (3.7%)
km (3.5%)
hora (2.1%)
kilometro (2.0%)

Cluster veiculo_35 (unidade): 
Tamanho: 1104
Descrições únicas: 911
20 Termos mais frequentes: ['veiculo', 'minimo', 'porta', 'ano', '1', '4', 'km', 'eletrico', '0', 'com', 'cor', '5', 'direcao', 'hidraulico', 'branco', 'modelo', 'motor', 'capacidade', 'ar', 'zero']
Descrições mais frequentes:
veiculo novo zero quilometro capacidade para cinco passageiro quatro porta cor branco motor 1 0 flex fabricacao 2014 2015 (1.0%)
veiculo popular zero quilometro 4 quatro porta direcao hidraulico flex cor branco para programa bolsa familia (0.7%)
veiculo automotor transporte passageiro com seguinte caracteristico ano 2013 modelo zero km 5 cinco lugar incluido motorista cor solido branco motor 1 0 refrigeracao agua 4 cilindro linha injecao eletronico potencia minimo 78 cv 8 oito valvula transmissao manual quatro marcha frente